In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

mathieugodbout_oct_postsurgery_visual_improvement_path = kagglehub.dataset_download('mathieugodbout/oct-postsurgery-visual-improvement')
muhammadusamazaman_sample_path = kagglehub.dataset_download('muhammadusamazaman/sample')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
df =pd.read_csv("/kaggle/input/oct-postsurgery-visual-improvement/oct_data/train/clinical_data.csv")



In [ ]:
df.head(10)

In [ ]:
df.info()
df.describe()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()


In [ ]:
sns.histplot(df["age"], kde=True)
plt.title("Age Distribution")
plt.show()


In [ ]:
sns.histplot(df["mh_size"], kde=True)
plt.title("Macular Hole Size Distribution")
plt.show()


In [ ]:
plt.figure(figsize=(12,6))
sns.heatmap(df.corr(), annot=True, cmap="coolwarm")
plt.title("Correlation Heatmap")
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


for column in ['age', 'mh_duration', 'mh_size', 'VA_baseline', 'VA_2weeks', 'VA_3months', 'VA_6months', 'VA_12months']:
    plt.figure(figsize=(6, 4))
    sns.histplot(df[column], kde=True)
    plt.title(f'Distribution of {column}')
    plt.show()


In [ ]:
!pip -q install tifffile

**Step 1: Define the Encoders (Fundus & OCT)**

In [ ]:
# import os, glob
# import numpy as np
# import pandas as pd
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader
# import tifffile as tiff
# from torchvision.utils import save_image

# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# print("DEVICE:", DEVICE)

# DATASET_ROOT = "/kaggle/input/oct-postsurgery-visual-improvement"  # adjust if Kaggle shows slightly different name
# TRAIN_DIR = os.path.join(DATASET_ROOT, "oct_data", "train", "octs")
# VAL_DIR   = os.path.join(DATASET_ROOT, "oct_data", "val", "octs")
# TEST_DIR  = os.path.join(DATASET_ROOT, "oct_data", "test", "octs")

# META_PATH = os.path.join(DATASET_ROOT, "oct_data","train" ,"clinical_data.csv")  # should exist in Kaggle dataset

# print("Train dir exists:", os.path.exists(TRAIN_DIR))

# print("Meta exists:", os.path.exists(META_PATH))


In [ ]:
# =========================
# Cell 1: Setup + Paths
# =========================
!pip -q install tifffile

import os, glob, re, random
import numpy as np
import pandas as pd
import tifffile as tiff

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import save_image

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("DEVICE:", DEVICE)

# Kaggle dataset root (as in your screenshot)
DATASET_ROOT = "/kaggle/input/oct-postsurgery-visual-improvement"
TRAIN_DIR = os.path.join(DATASET_ROOT, "oct_data", "train", "octs")
VAL_DIR   = os.path.join(DATASET_ROOT, "oct_data", "val", "octs")
TEST_DIR  = os.path.join(DATASET_ROOT, "oct_data", "test", "octs")

META_PATH = os.path.join(DATASET_ROOT, "clinical_data.csv")  # FIXED

print("Train:", os.path.exists(TRAIN_DIR), TRAIN_DIR)
print("Val  :", os.path.exists(VAL_DIR), VAL_DIR)
print("Meta :", os.path.exists(META_PATH), META_PATH)

# Hyperparams
IMG_SIZE = 256
BATCH_SIZE = 4
EPOCHS = 20
LR_G = 2e-4
LR_D = 2e-4
Z_DIM = 128

LAMBDA_L1   = 10.0
LAMBDA_SSIM = 2.0
LAMBDA_FREQ = 1.0

OUT_DIR = "/kaggle/working/hmdf_outputs"
os.makedirs(OUT_DIR, exist_ok=True)

def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
set_seed(42)


In [ ]:
# =========================
# Cell 2: Losses + FFT + SSIM
# =========================
def hinge_d_loss(d_real, d_fake):
    return (F.relu(1.0 - d_real).mean() + F.relu(1.0 + d_fake).mean())

def hinge_g_loss(d_fake):
    return (-d_fake).mean()

def fft_mag(x: torch.Tensor) -> torch.Tensor:
    X = torch.fft.fft2(x, norm="ortho")
    return torch.sqrt(X.real**2 + X.imag**2 + 1e-8)

def ssim_loss(x: torch.Tensor, y: torch.Tensor, C1=0.01**2, C2=0.03**2) -> torch.Tensor:
    mu_x = F.avg_pool2d(x, 3, 1, 1)
    mu_y = F.avg_pool2d(y, 3, 1, 1)
    sigma_x = F.avg_pool2d(x * x, 3, 1, 1) - mu_x * mu_x
    sigma_y = F.avg_pool2d(y * y, 3, 1, 1) - mu_y * mu_y
    sigma_xy = F.avg_pool2d(x * y, 3, 1, 1) - mu_x * mu_y

    ssim_map = ((2 * mu_x * mu_y + C1) * (2 * sigma_xy + C2)) / (
        (mu_x * mu_x + mu_y * mu_y + C1) * (sigma_x + sigma_y + C2) + 1e-8
    )
    return 1.0 - ssim_map.mean()


In [ ]:
!pip install imagecodecs

In [ ]:
# =========================
# Cell 3: Dataset (TIFF) + Metadata + Timepoints (FIXED)
# =========================
TIMEPOINTS = ["baseline", "2weeks", "3months", "6months", "12months"]

def parse_filename(path: str):
    # Examples:
    # 104_baseline_H.tiff
    # 104_6months_V.tiff
    base = os.path.basename(path)
    name = os.path.splitext(base)[0]
    parts = name.split("_")
    pid = int(parts[0])
    tp = parts[1].lower() if len(parts) > 1 else "baseline"
    orient = parts[2].upper() if len(parts) > 2 else "H"
    if tp not in TIMEPOINTS:
        # fallback: try to detect
        for t in TIMEPOINTS:
            if t in name.lower():
                tp = t
                break
    return pid, tp, orient

class OCTDataset(Dataset):
    def __init__(self, folder, meta_path=None, img_size=256, use_meta=True):
        self.files = sorted(glob.glob(os.path.join(folder, "*.tiff")))
        self.img_size = img_size
        self.use_meta = use_meta and (meta_path is not None) and os.path.exists(meta_path)

        self.meta = None
        self.cond_cols = ["age","sex","pseudophakic","mh_duration","elevated_edge","mh_size","VA_baseline"]

        if self.use_meta:
            self.meta = pd.read_csv(meta_path).set_index("id")
            self.cond_cols = [c for c in self.cond_cols if c in self.meta.columns]

    def __len__(self):
        return len(self.files)

    def _read_tiff_norm(self, path):
        arr = tiff.imread(path)
        if arr.ndim == 3:
            arr = arr[..., 0]
        arr = arr.astype(np.float32)
        arr = (arr - arr.min()) / ((arr.max() - arr.min()) + 1e-8)  # [0,1]
        x = torch.from_numpy(arr).unsqueeze(0).unsqueeze(0)          # (1,1,H,W)
        x = F.interpolate(x, size=(self.img_size, self.img_size), mode="bilinear", align_corners=False)
        x = x.squeeze(0)                                             # (1,H,W)
        x = x * 2.0 - 1.0                                            # [-1,1]
        return x

    def __getitem__(self, idx):
        path = self.files[idx]
        pid, tp, orient = parse_filename(path)
        img = self._read_tiff_norm(path)

        # timepoint one-hot
        tp_vec = torch.zeros(len(TIMEPOINTS), dtype=torch.float32)
        if tp in TIMEPOINTS:
            tp_vec[TIMEPOINTS.index(tp)] = 1.0

        if self.use_meta and (pid in self.meta.index):
            row = self.meta.loc[pid]
            vals = []
            for c in self.cond_cols:
                v = row[c]
                if pd.isna(v): v = 0.0
                vals.append(float(v))
            cond = torch.tensor(vals, dtype=torch.float32)
            # stable standardization
            cond = (cond - cond.mean()) / (cond.std() + 1e-6)
            cond = torch.cat([cond, tp_vec], dim=0)
        else:
            cond = tp_vec

        return {"img": img, "cond": cond, "pid": pid, "tp": tp, "path": path}

ds_train = OCTDataset(TRAIN_DIR, META_PATH, img_size=IMG_SIZE, use_meta=True)
ds_val   = OCTDataset(VAL_DIR, META_PATH, img_size=IMG_SIZE, use_meta=True)

print("Train samples:", len(ds_train))
print("Val samples  :", len(ds_val))
s = ds_train[0]
print("Example:", s["pid"], s["tp"], s["img"].shape, s["cond"].shape)

train_loader = DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, drop_last=True)
val_loader   = DataLoader(ds_val, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, drop_last=True)

COND_DIM = ds_train[0]["cond"].numel()
print("COND_DIM:", COND_DIM)


In [ ]:
# =========================
# Cell 4: Blocks + DWT (FIXED) + Encoders (OCT + Optional Fundus Stub)
# =========================
class ConvBNAct(nn.Module):
    def __init__(self, in_ch, out_ch, k=3, s=1, p=1, act=True):
        super().__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, k, s, p, bias=False)
        self.bn = nn.BatchNorm2d(out_ch)
        self.act = nn.SiLU(inplace=True) if act else nn.Identity()
    def forward(self, x):
        return self.act(self.bn(self.conv(x)))

class ResBlock(nn.Module):
    def __init__(self, ch):
        super().__init__()
        self.c1 = ConvBNAct(ch, ch, 3, 1, 1)
        self.c2 = ConvBNAct(ch, ch, 3, 1, 1, act=False)
    def forward(self, x):
        return F.silu(x + self.c2(self.c1(x)))

class HaarDWT(nn.Module):
    """
    Correct grouped-conv Haar DWT
    Input : (B,C,H,W)
    Output: (B,4C,H/2,W/2) bands concatenated per-channel
    """
    def __init__(self):
        super().__init__()
        ll = torch.tensor([[0.5, 0.5],[0.5, 0.5]], dtype=torch.float32)
        lh = torch.tensor([[0.5, 0.5],[-0.5,-0.5]], dtype=torch.float32)
        hl = torch.tensor([[0.5,-0.5],[0.5,-0.5]], dtype=torch.float32)
        hh = torch.tensor([[0.5,-0.5],[-0.5,0.5]], dtype=torch.float32)
        filt = torch.stack([ll, lh, hl, hh], dim=0)  # (4,2,2)
        self.register_buffer("filt", filt)

    def forward(self, x):
        B, C, H, W = x.shape
        filt = self.filt.unsqueeze(1)                 # (4,1,2,2)
        weight = filt.repeat(C, 1, 1, 1)              # (4C,1,2,2)
        y = F.conv2d(x, weight, stride=2, padding=0, groups=C)  # (B,4C,H/2,W/2)
        return y

class OCTEncoder(nn.Module):
    def __init__(self, in_ch=1, base=64, feat_ch=256):
        super().__init__()
        self.spatial = nn.Sequential(
            ConvBNAct(in_ch, base, 5, 2, 2),      # 128
            ConvBNAct(base, base, 3, 1, 1),
            ConvBNAct(base, base*2, 3, 2, 1),     # 64
            ResBlock(base*2),
            ResBlock(base*2),
            ConvBNAct(base*2, feat_ch, 1, 1, 0)
        )
        self.dwt = HaarDWT()
        self.freq_proj = nn.Sequential(
            ConvBNAct(feat_ch*4, feat_ch, 1, 1, 0)
        )

    def forward(self, x):
        f_sp = self.spatial(x)                         # (B,256,64,64)
        f_dwt = self.dwt(f_sp)                         # (B,1024,32,32)
        f_dwt = F.interpolate(f_dwt, size=f_sp.shape[-2:], mode="bilinear", align_corners=False)
        f_fr = self.freq_proj(f_dwt)                   # (B,256,64,64)
        return f_sp, f_fr

class FundusEncoderStub(nn.Module):
    """
    Kept for architectural compatibility (inactive if no fundus data).
    """
    def __init__(self):
        super().__init__()
    def forward(self, x):
        raise RuntimeError("Fundus encoder is inactive in OCT-only training.")


In [ ]:
# =========================
# Cell 5: HFSF + Generator + Discriminators
# =========================
class HFSFBlock(nn.Module):
    def __init__(self, ch=256, out_ch=512):
        super().__init__()
        self.gate = nn.Sequential(nn.Conv2d(ch, ch, 1, 1, 0), nn.Sigmoid())
        self.freq_mix = nn.Sequential(
            ConvBNAct(ch, ch, 1, 1, 0),
            ConvBNAct(ch, ch, 3, 1, 1),
        )
        self.harm = nn.Sequential(
            nn.Conv2d(ch, out_ch, 1, 1, 0, bias=False),
            nn.GroupNorm(8, out_ch),
            nn.SiLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, 1, 1, groups=out_ch, bias=False),
            nn.Conv2d(out_ch, out_ch, 1, 1, 0, bias=False),
        )

    def forward(self, f_sp, f_fr):
        g = self.gate(f_sp)
        f_fft = fft_mag(f_sp)
        f_freq = self.freq_mix(f_fr + f_fft)
        fused = g * f_freq + (1.0 - g) * f_sp
        return self.harm(fused)

class MappingNet(nn.Module):
    def __init__(self, in_dim, out_dim=512):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, out_dim),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(out_dim, out_dim),
            nn.LeakyReLU(0.2, inplace=True),
        )
    def forward(self, p):
        return self.net(p)

class Generator(nn.Module):
    def __init__(self, fused_ch=512, z_dim=128, cond_dim=0, out_ch=1):
        super().__init__()
        self.use_cond = cond_dim > 0
        self.map_p = MappingNet(cond_dim, 512) if self.use_cond else None
        self.z_proj = nn.Linear(z_dim, 512)

        self.dec1 = ConvBNAct(fused_ch + 512, 512, 3, 1, 1)
        self.dec2 = ConvBNAct(512, 256, 3, 1, 1)

        self.up1 = nn.Sequential(
            nn.Upsample(scale_factor=2, mode="bilinear", align_corners=False),  # 128
            ConvBNAct(256, 128, 3, 1, 1),
            ResBlock(128)
        )
        self.up2 = nn.Sequential(
            nn.Upsample(scale_factor=2, mode="bilinear", align_corners=False),  # 256
            ConvBNAct(128, 64, 3, 1, 1),
            ResBlock(64)
        )
        self.to_img = nn.Conv2d(64, out_ch, 1, 1, 0)

    def forward(self, fused_feat, z, p=None):
        B, _, H, W = fused_feat.shape
        zc = self.z_proj(z)
        style = zc
        if self.use_cond and p is not None:
            style = style + self.map_p(p)
        style_map = style.view(B, 512, 1, 1).expand(B, 512, H, W)
        x = torch.cat([fused_feat, style_map], dim=1)
        x = self.dec2(self.dec1(x))
        x = self.up1(x)
        x = self.up2(x)
        return torch.tanh(self.to_img(x))

class PatchDiscriminator(nn.Module):
    def __init__(self, in_ch=1, base=64):
        super().__init__()
        def block(ic, oc, s):
            return nn.Sequential(
                nn.utils.spectral_norm(nn.Conv2d(ic, oc, 4, s, 1)),
                nn.LeakyReLU(0.2, inplace=True),
            )
        self.net = nn.Sequential(
            block(in_ch, base, 2),
            block(base, base*2, 2),
            block(base*2, base*4, 2),
            block(base*4, base*8, 1),
            nn.utils.spectral_norm(nn.Conv2d(base*8, 1, 4, 1, 1))
        )
    def forward(self, x):
        return self.net(x)

class FrequencyDiscriminator(nn.Module):
    def __init__(self, in_ch=1, base=32):
        super().__init__()
        self.net = nn.Sequential(
            nn.utils.spectral_norm(nn.Conv2d(in_ch, base, 3, 2, 1)),
            nn.LeakyReLU(0.2, inplace=True),
            nn.utils.spectral_norm(nn.Conv2d(base, base*2, 3, 2, 1)),
            nn.LeakyReLU(0.2, inplace=True),
            nn.utils.spectral_norm(nn.Conv2d(base*2, base*4, 3, 2, 1)),
            nn.LeakyReLU(0.2, inplace=True),
            nn.utils.spectral_norm(nn.Conv2d(base*4, 1, 3, 1, 1)),
        )
    def forward(self, x):
        return self.net(fft_mag(x))


In [ ]:
# =========================
# Cell 6: Init Models + Training Loop (OCT-only Augmentation)
# =========================
enc  = OCTEncoder(in_ch=1).to(DEVICE)
hfsf = HFSFBlock(ch=256, out_ch=512).to(DEVICE)
gen  = Generator(fused_ch=512, z_dim=Z_DIM, cond_dim=COND_DIM, out_ch=1).to(DEVICE)

d_anat = PatchDiscriminator(in_ch=1).to(DEVICE)
d_freq = FrequencyDiscriminator(in_ch=1).to(DEVICE)

opt_g = torch.optim.Adam(list(enc.parameters()) + list(hfsf.parameters()) + list(gen.parameters()),
                         lr=LR_G, betas=(0.5, 0.999))
opt_d = torch.optim.Adam(list(d_anat.parameters()) + list(d_freq.parameters()),
                         lr=LR_D, betas=(0.5, 0.999))

scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE == "cuda"))

fixed = next(iter(val_loader))
fixed_real = fixed["img"].to(DEVICE)
fixed_cond = fixed["cond"].to(DEVICE)
fixed_z = torch.randn(fixed_real.size(0), Z_DIM, device=DEVICE)

for epoch in range(1, EPOCHS + 1):
    enc.train(); hfsf.train(); gen.train()
    d_anat.train(); d_freq.train()

    for step, batch in enumerate(train_loader, start=1):
        real = batch["img"].to(DEVICE)      # (B,1,256,256) [-1,1]
        cond = batch["cond"].to(DEVICE)     # (B,COND_DIM)

        # ---- Train D ----
        z = torch.randn(real.size(0), Z_DIM, device=DEVICE)
        with torch.cuda.amp.autocast(enabled=(DEVICE == "cuda")):
            f_sp, f_fr = enc(real)
            fused = hfsf(f_sp, f_fr)
            fake = gen(fused, z, cond).detach()

            loss_d = hinge_d_loss(d_anat(real), d_anat(fake)) + hinge_d_loss(d_freq(real), d_freq(fake))

        opt_d.zero_grad(set_to_none=True)
        scaler.scale(loss_d).backward()
        scaler.step(opt_d)

        # ---- Train G (+ Encoder + HFSF) ----
        z = torch.randn(real.size(0), Z_DIM, device=DEVICE)
        with torch.cuda.amp.autocast(enabled=(DEVICE == "cuda")):
            f_sp, f_fr = enc(real)
            fused = hfsf(f_sp, f_fr)
            fake = gen(fused, z, cond)

            adv   = hinge_g_loss(d_anat(fake)) + hinge_g_loss(d_freq(fake))
            l1    = F.l1_loss(fake, real)
            lssim = ssim_loss(fake, real)
            lfreq = F.l1_loss(fft_mag(fake), fft_mag(real))

            loss_g = adv + (LAMBDA_L1 * l1) + (LAMBDA_SSIM * lssim) + (LAMBDA_FREQ * lfreq)

        opt_g.zero_grad(set_to_none=True)
        scaler.scale(loss_g).backward()
        scaler.step(opt_g)
        scaler.update()

        if step % 50 == 0:
            print(f"Epoch {epoch}/{EPOCHS} Step {step} | D {loss_d.item():.4f} | G {loss_g.item():.4f} | adv {adv.item():.4f}")

    # Save samples each epoch
    enc.eval(); hfsf.eval(); gen.eval()
    with torch.no_grad():
        f_sp, f_fr = enc(fixed_real)
        fused = hfsf(f_sp, f_fr)
        fake = gen(fused, fixed_z, fixed_cond)

        grid = torch.cat([fixed_real, fake], dim=0)
        save_image((grid + 1) / 2, os.path.join(OUT_DIR, f"samples_epoch_{epoch}.png"), nrow=fixed_real.size(0))

    torch.save({
        "enc": enc.state_dict(),
        "hfsf": hfsf.state_dict(),
        "gen": gen.state_dict(),
        "d_anat": d_anat.state_dict(),
        "d_freq": d_freq.state_dict(),
        "epoch": epoch,
    }, os.path.join(OUT_DIR, f"ckpt_epoch_{epoch}.pt"))

    print("Saved:", f"samples_epoch_{epoch}.png")


In [ ]:
# # =========================
# # Cell 7 (FINAL): Load ckpt_epoch_18 + Augmentation Generation
# # =========================
# import os, glob
# import numpy as np
# import pandas as pd
# import torch
# import torch.nn.functional as F
# import tifffile as tiff

# # --------- 1) Load Checkpoint (Epoch 18) ----------
# CKPT_PATH = "/kaggle/working/hmdf_outputs/ckpt_epoch_18.pt"

# ckpt = torch.load(CKPT_PATH, map_location=DEVICE)
# enc.load_state_dict(ckpt["enc"])
# hfsf.load_state_dict(ckpt["hfsf"])
# gen.load_state_dict(ckpt["gen"])

# enc.eval(); hfsf.eval(); gen.eval()
# print("Loaded checkpoint:", CKPT_PATH)

# # --------- 2) Output folder ----------
# AUG_DIR = "/kaggle/working/augmented_octs_epoch18"
# os.makedirs(AUG_DIR, exist_ok=True)

# # --------- 3) Augmentation settings ----------
# N_SYN_PER_REAL = 5
# MAX_REAL = 2   # set 2 if you want only 2 images demo
# SOURCE_DIR = TRAIN_DIR  # or VAL_DIR / TEST_DIR

# real_paths = sorted(glob.glob(os.path.join(SOURCE_DIR, "*.tiff")))[:MAX_REAL]
# print("Total real images selected:", len(real_paths))

# def save_tiff_from_tensor(x01: torch.Tensor, path: str):
#     # x01: (H,W) in [0,1]
#     arr = (x01.clamp(0,1).cpu().numpy() * 255).astype(np.uint8)
#     tiff.imwrite(path, arr)

# # --------- 4) Generate synthetic images ----------
# for pth in real_paths:
#     pid, tp, orient = parse_filename(pth)

#     # Load TIFF exactly like dataset
#     arr = tiff.imread(pth)
#     if arr.ndim == 3:
#         arr = arr[..., 0]
#     arr = arr.astype(np.float32)
#     arr = (arr - arr.min()) / ((arr.max() - arr.min()) + 1e-8)

#     x = torch.from_numpy(arr).unsqueeze(0).unsqueeze(0)  # (1,1,H,W)
#     x = F.interpolate(x, size=(IMG_SIZE, IMG_SIZE), mode="bilinear", align_corners=False)
#     x = x.squeeze(0)                                     # (1,256,256)
#     x = x * 2 - 1                                        # [-1,1]
#     xB = x.unsqueeze(0).to(DEVICE)                       # (1,1,256,256)

#     # Condition vector
#     tp_vec = torch.zeros(len(TIMEPOINTS), dtype=torch.float32)
#     if tp in TIMEPOINTS:
#         tp_vec[TIMEPOINTS.index(tp)] = 1.0

#     if ds_train.meta is not None and pid in ds_train.meta.index:
#         row = ds_train.meta.loc[pid]
#         vals = []
#         for c in ds_train.cond_cols:
#             v = row[c]
#             if pd.isna(v):
#                 v = 0.0
#             vals.append(float(v))
#         cvec = torch.tensor(vals, dtype=torch.float32)
#         cvec = (cvec - cvec.mean()) / (cvec.std() + 1e-6)
#         cond = torch.cat([cvec, tp_vec], dim=0)
#     else:
#         cond = tp_vec

#     cond = cond.unsqueeze(0).to(DEVICE)  # (1,COND_DIM)

#     with torch.no_grad():
#         f_sp, f_fr = enc(xB)
#         fused = hfsf(f_sp, f_fr)

#         for k in range(N_SYN_PER_REAL):
#             z = torch.randn(1, Z_DIM, device=DEVICE)
#             fake = gen(fused, z, cond)  # (1,1,256,256) in [-1,1]
#             fake01 = (fake.squeeze(0).squeeze(0) + 1) / 2  # (256,256) in [0,1]

#             out_name = f"{pid}_{tp}_{orient}_syn{k+1}_E18.tiff"
#             out_path = os.path.join(AUG_DIR, out_name)
#             save_tiff_from_tensor(fake01, out_path)

# print("Saved augmented TIFFs in:", AUG_DIR)
# print("Count:", len(os.listdir(AUG_DIR)))


In [ ]:
# =========================
# Cell 7 (FINAL): Augmentation from 2 TEST images (Patient 104)
# =========================
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import tifffile as tiff

# --------- Load checkpoint ----------
CKPT_PATH = "/kaggle/working/hmdf_outputs/ckpt_epoch_18.pt"
ckpt = torch.load(CKPT_PATH, map_location=DEVICE)

enc.load_state_dict(ckpt["enc"])
hfsf.load_state_dict(ckpt["hfsf"])
gen.load_state_dict(ckpt["gen"])

enc.eval(); hfsf.eval(); gen.eval()
print("Loaded checkpoint:", CKPT_PATH)

# --------- Output folder ----------
AUG_DIR = "/kaggle/working/augmented_img_final"
os.makedirs(AUG_DIR, exist_ok=True)


real_paths = sorted(glob.glob("/kaggle/input/oct-postsurgery-visual-improvement/oct_data/test/octs/*.tiff"))[:98]

N_SYN_PER_REAL = 1

def save_tiff_from_tensor(x01, path):
    arr = (x01.clamp(0,1).cpu().numpy() * 255).astype(np.uint8)
    tiff.imwrite(path, arr)

# --------- Augmentation loop ----------
for pth in real_paths:
    pid, tp, orient = parse_filename(pth)

    arr = tiff.imread(pth)
    if arr.ndim == 3:
        arr = arr[..., 0]

    arr = arr.astype(np.float32)
    arr = (arr - arr.min()) / ((arr.max() - arr.min()) + 1e-8)

    x = torch.from_numpy(arr).unsqueeze(0).unsqueeze(0)
    x = F.interpolate(x, size=(IMG_SIZE, IMG_SIZE), mode="bilinear", align_corners=False)
    x = x.squeeze(0)
    x = x * 2 - 1
    xB = x.unsqueeze(0).to(DEVICE)

    # ---- condition vector ----
    tp_vec = torch.zeros(len(TIMEPOINTS))
    if tp in TIMEPOINTS:
        tp_vec[TIMEPOINTS.index(tp)] = 1.0

    if ds_train.meta is not None and pid in ds_train.meta.index:
        row = ds_train.meta.loc[pid]
        vals = []
        for c in ds_train.cond_cols:
            v = row[c]
            if pd.isna(v): v = 0.0
            vals.append(float(v))
        cvec = torch.tensor(vals)
        cvec = (cvec - cvec.mean()) / (cvec.std() + 1e-6)
        cond = torch.cat([cvec, tp_vec], dim=0)
    else:
        cond = tp_vec

    cond = cond.unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        f_sp, f_fr = enc(xB)
        fused = hfsf(f_sp, f_fr)

        for k in range(N_SYN_PER_REAL):
            z = torch.randn(1, Z_DIM, device=DEVICE)
            fake = gen(fused, z, cond)
            fake01 = (fake.squeeze() + 1) / 2

            out_name = f"{pid}_{tp}_{orient}_syn{k+1}_E18.tiff"
            save_tiff_from_tensor(fake01, os.path.join(AUG_DIR, out_name))

print("Augmented images saved in:", AUG_DIR)
print("Total:", len(os.listdir(AUG_DIR)))


In [ ]:
# =========================
# Cell 8: Side-by-Side Comparison (Original vs Synthetic)
# =========================
import matplotlib.pyplot as plt
import tifffile as tiff
import os
import numpy as np

# -------- Paths --------
ORIGINAL_PATH = "/kaggle/input/oct-postsurgery-visual-improvement/oct_data/test/octs/104_2weeks_H.tiff"
SYN_DIR = "/kaggle/working/augmented_patient_104"

# Select synthetic images to compare
SYN_FILES = [
    "104_2weeks_H_syn1_E18.tiff",
    "104_2weeks_H_syn2_E18.tiff",
    "104_2weeks_H_syn3_E18.tiff",
    "104_2weeks_H_syn6_E18.tiff",
    "104_2weeks_H_syn10_E18.tiff",
]

# -------- Load images --------
orig = tiff.imread(ORIGINAL_PATH)
if orig.ndim == 3:
    orig = orig[..., 0]

syn_images = []
for f in SYN_FILES:
    p = os.path.join(SYN_DIR, f)
    img = tiff.imread(p)
    syn_images.append(img)

# -------- Plot --------
n_cols = len(syn_images) + 1
plt.figure(figsize=(3 * n_cols, 4))

# Original
plt.subplot(1, n_cols, 1)
plt.imshow(orig, cmap="gray")
plt.title("Original OCT\n(104 | 2 weeks | H)")
plt.axis("off")

# Synthetic
for i, img in enumerate(syn_images):
    plt.subplot(1, n_cols, i + 2)
    plt.imshow(img, cmap="gray")
    plt.title(f"Synthetic {i+1}\n(Epoch 18)")
    plt.axis("off")

plt.suptitle("Side-by-Side Comparison: Original vs Synthetic OCT Images", fontsize=14)
plt.tight_layout()
plt.show()


In [ ]:
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import cv2
import numpy as np
import glob
import os

REAL_DIR = "/kaggle/input/oct-postsurgery-visual-improvement/oct_data/test/octs"
SYN_DIR  = "/kaggle/working/augmented_patient_104"

ssim_scores, psnr_scores = [], []

real_imgs = sorted(glob.glob(os.path.join(REAL_DIR, "*.tiff")))
syn_imgs  = sorted(glob.glob(os.path.join(SYN_DIR, "*.tiff")))

for r, s in zip(real_imgs, syn_imgs):
    real = cv2.imread(r, cv2.IMREAD_GRAYSCALE)
    syn  = cv2.imread(s, cv2.IMREAD_GRAYSCALE)

    real = cv2.resize(real, (256,256))
    syn  = cv2.resize(syn, (256,256))

    ssim_scores.append(ssim(real, syn, data_range=255))
    psnr_scores.append(psnr(real, syn, data_range=255))

print("SSIM mean:", np.mean(ssim_scores))
print("PSNR mean:", np.mean(psnr_scores))


In [ ]:
import os, glob
import numpy as np
import tifffile as tiff
import torch
import torch.nn.functional as F
from torchvision.models import inception_v3, Inception_V3_Weights
from scipy.linalg import sqrtm

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

REAL_DIR = "/kaggle/input/oct-postsurgery-visual-improvement/oct_data/test/octs"
SYN_DIR  = "/kaggle/working/augmented_img_final"

# ---- choose real images ----
# Option A: only patient 104 (recommended for consistency)
real_paths = sorted(glob.glob(os.path.join(REAL_DIR, "*.tiff")))

# Option B: If you want more real images for stable FID, use more patients:
# real_paths = sorted(glob.glob(os.path.join(REAL_DIR, "*.tiff")))[:50]

syn_paths  = sorted(glob.glob(os.path.join(SYN_DIR, "*.tiff")))

print("Real images:", len(real_paths))
print("Synthetic images:", len(syn_paths))

assert len(real_paths) > 0, "No real images found!"
assert len(syn_paths)  > 0, "No synthetic images found!"

# ---- Load and preprocess for Inception (299x299, RGB) ----
def load_for_inception(paths, max_images=50):
    imgs = []
    for p in paths[:max_images]:
        img = tiff.imread(p)
        if img.ndim == 3:
            img = img[..., 0]
        img = img.astype(np.float32)
        img = (img - img.min()) / (img.max() - img.min() + 1e-8)  # [0,1]
        img = torch.from_numpy(img).unsqueeze(0)  # (1,H,W)

        img = img.unsqueeze(0)  # (1,1,H,W)
        img = F.interpolate(img, size=(299, 299), mode="bilinear", align_corners=False)
        img = img.squeeze(0)    # (1,299,299)

        img = img.repeat(3, 1, 1)  # (3,299,299) grayscale -> RGB
        imgs.append(img)
    return torch.stack(imgs, dim=0)  # (N,3,299,299)

# ---- Inception feature extractor ----
from torchvision.models import inception_v3, Inception_V3_Weights

weights = Inception_V3_Weights.DEFAULT
inception = inception_v3(weights=weights)   # aux_logits default = True
inception.fc = torch.nn.Identity()
inception = inception.to(DEVICE).eval()

@torch.no_grad()
def get_feats(batch_imgs, batch_size=8):
    feats = []
    for i in range(0, batch_imgs.size(0), batch_size):
        x = batch_imgs[i:i+batch_size].to(DEVICE)
        f = inception(x)
        feats.append(f.cpu().numpy())
    return np.concatenate(feats, axis=0)

# ---- FID computation ----
def fid_score(real_feats, gen_feats):
    mu_r, mu_g = real_feats.mean(axis=0), gen_feats.mean(axis=0)
    sig_r = np.cov(real_feats, rowvar=False)
    sig_g = np.cov(gen_feats, rowvar=False)

    covmean = sqrtm(sig_r @ sig_g)
    if np.iscomplexobj(covmean):
        covmean = covmean.real

    diff = mu_r - mu_g
    fid = diff @ diff + np.trace(sig_r + sig_g - 2 * covmean)
    return float(fid)

# ---- Run ----
real_imgs = load_for_inception(real_paths, max_images=50)
syn_imgs  = load_for_inception(syn_paths,  max_images=50)

real_feats = get_feats(real_imgs)
syn_feats  = get_feats(syn_imgs)

fid = fid_score(real_feats, syn_feats)
print("✅ FID:", fid)
